<a href="https://colab.research.google.com/github/Gusta02/Bandas_de_Bollinger/blob/main/Bandas_de_Bollinger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 6.4 MB 31.9 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

In [ ]:
#pegar a ação que iremos trabalhar

acao = yf.Ticker("NTCO3.SA")
data = acao.history(period='1y')
df = data[['Close']]

In [ ]:
df

,Close
Date,
2021-02-09,51.090000
2021-02-10,50.720001
2021-02-11,53.759998
2021-02-12,53.299999
2021-02-17,53.090000
...,...
2022-02-03,21.450001
2022-02-04,21.010000
2022-02-07,20.730000


In [ ]:
#Calcular a média movel
mm = df.rolling(window=20).mean()
mm

,Close
Date,
2021-02-09,NaN
2021-02-10,NaN
2021-02-11,NaN
2021-02-12,NaN
2021-02-17,NaN
...,...
2022-02-03,22.2335
2022-02-04,22.1785
2022-02-07,22.1335


In [ ]:
# Calculo do desvio padrão

dpm = df.rolling(window=20).std()
dpm

,Close
Date,
2021-02-09,NaN
2021-02-10,NaN
2021-02-11,NaN
2021-02-12,NaN
2021-02-17,NaN
...,...
2022-02-03,0.929835
2022-02-04,0.969223
2022-02-07,1.015803


In [ ]:
# calculo da banda superior e inferior

sup_band = mm + 2 + dpm
inf_band = mm - 2 +dpm

In [ ]:
#alterar nome das colunas de banda sup e banda inf

sup_band = sup_band.rename(columns = {'Close': 'superior'})
inf_band = inf_band.rename(columns = {'Close' : 'inferior'})

In [ ]:
#unir as colunas

bandas_bolliger = df.join( sup_band ).join( inf_band )

In [ ]:
bandas_bolliger

,Close,superior,inferior
Date,,,
2021-02-09,51.090000,NaN,NaN
2021-02-10,50.720001,NaN,NaN
2021-02-11,53.759998,NaN,NaN
2021-02-12,53.299999,NaN,NaN
2021-02-17,53.090000,NaN,NaN
...,...,...,...
2022-02-03,21.450001,25.163335,21.163335
2022-02-04,21.010000,25.147723,21.147723
2022-02-07,20.730000,25.149303,21.149303


In [ ]:
#Retirar os nulos

bandas_bolliger.dropna(inplace=True)

In [ ]:
#calculo dos pontos de venda e compra

compra = bandas_bolliger[bandas_bolliger['Close'] <= bandas_bolliger['inferior']]
venda = bandas_bolliger[bandas_bolliger['Close'] >= bandas_bolliger['superior']]

In [ ]:
#plantar no gráfico

import plotly.io as pio
import plotly.graph_objects as go

In [ ]:
pio.templates.default = 'plotly_dark'

fig = go.Figure()
fig.add_trace(go.Scatter(x=inf_band.index,
                         y=inf_band['inferior'],
                         name = "Banda Inferior",
                         line_color = 'rgba(173,204,255,0.2)'
                         ))

fig.add_trace(go.Scatter(x=sup_band.index,
                         y=sup_band['superior'],
                         name = "Banda Superior",
                         fill = 'tonexty',
                         fillcolor = 'rgba(173,204,255,0.2)',
                         line_color = 'rgba(173,204,255,0.2)'
                         ))

fig.add_trace(go.Scatter(x=df.index,
                         y=df['Close'],
                         name='Preço Fechamento',
                         line_color = '#636EFA'
                         ))

fig.add_trace(go.Scatter(x=mm.index,
                         y=mm['Close'],
                         name='Média Movel',
                         line_color = '#FEC852'
                         ))

fig.add_trace(go.Scatter(x=compra.index,
                         y=compra['Close'],
                         name='Compra',
                         mode='markers',
                         marker=dict(
                             color='#00CC96',
                             size=8,
                             )
                         ))

fig.add_trace(go.Scatter(x=venda.index,
                         y=venda['Close'],
                         name='Venda',
                         mode='markers',
                         marker=dict(
                             color='#EF553B',
                             size=8,
                             )
                         ))
fig.show()